# Central Line UNet


This noteboook contains all the code neccesary to train the UNet used in this project for ultrasound image segmentation.

Note that it assumes [SlicerIGT/aigt](https://github.com/SlicerIGT/aigt) is already cloned locally.

Things that must be changed to run this locally:
- Output_dir
- aigt_repo_path

---

# Define user parameters

In [ ]:
import numpy as np

#Paths
data_input_dir = r"C:\\Users\\cbarr\\OneDrive - Queen's University\\Grad School\\Courses\\CISC 881\\Project\\Data\\PilotNpData"
aigt_repo_path = r"C:\repos\aigt"
output_dir = r"C:\Users\cbarr\OneDrive - Queen's University\Grad School\Courses\CISC 881\Project\Testing_Folder"

#Notebook name
notebook_name = 'Central_Line_UNet'

#Sequences to be used for validation and testing; the rest is for training
test_idx = [0,1,2,3,4]
current_fold = "fold_1"

# Learning parameters
ultrasound_size = 128                  #Dimensions of the US images used
num_epochs = 1                         #Number of times to adjust weights following use of a batch of data
batch_size = 128                       #Number of images to feed in for each epoch
max_learning_rate = 0.02               #Used in defining Adam parameters
min_learning_rate = 0.00001            #Used to generate learning_rate_decay
regularization_rate = 0.0001           #Used in uNet definition; L1 bias regulation
filter_multiplier = 8                  #used in UNet definition
class_weights = np.array([0.1, 0.9])   #Weights for weighted categorical cross entropy
learning_rate_decay = (max_learning_rate - min_learning_rate) / num_epochs #the rate at which the learning rate decays; adam parameter

# Training data augmentation parameters for segmentation batch generation
max_shift_factor = 0.12
max_rotation_angle = 10
max_zoom_factor = 1.1
min_zoom_factor = 0.8

acceptable_margin_mm = 1.0 #For evaluation of segmentation
mm_per_pixel = 1.0

#Define roc thresholds
roc_thresholds = [0.9, 0.8, 0.7, 0.65, 0.6, 0.55, 0.5, 0.45, 0.4, 0.35, 0.3, 0.25, 0.2, 0.15, 0.1,
                  0.08, 0.06, 0.04, 0.02, 0.01,
                  0.008, 0.006, 0.004, 0.002, 0.001]

---

# Imports
## Python packages

In [ ]:
import os
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import datetime
from sklearn.model_selection import train_test_split
from random import sample

## Libraries in aigt repository

In [ ]:
sys.path.append(aigt_repo_path)
sys.path.append(os.path.join(aigt_repo_path, "UltrasoundSegmentation"))

import ultrasound_batch_generator as generator
import evaluation_metrics
import Models.segmentation_unet as unet
import utils

---

# Read in data

In [ ]:
#Read the US image and segmentations paths into lists
seg_files = list(sorted(Path(data_input_dir).glob("*_segmentation.npy")))
us_files = list(sorted(Path(data_input_dir).glob("*_ultrasound.npy")))

#Go through all file paths in both arrays and append to list
seg_data_by_seq = []
us_data_by_seq = []
for i in range(len(seg_files)):

    #Load the current files as 3D numpy arrays
    seg_np = np.load(os.path.abspath(seg_files[i]))
    us_np = np.load(os.path.abspath(us_files[i]))
    
    #Normalize and add channel dimension
    seg_np = seg_np[...,np.newaxis] / 255
    us_np = us_np[...,np.newaxis] / 255
    
    #Append to the collector lists.
    seg_data_by_seq.append(seg_np.transpose(2,1,0,3))
    us_data_by_seq.append(us_np.transpose(2,1,0,3))

#Verify that the total number of segmentation images matches the total number of us images.
for idx in range(len(seg_data_by_seq)):
    if len(seg_data_by_seq[idx][0]) != len(us_data_by_seq[idx][0]):
        print("Data Problem: Dataset {} has {} ultrasounds and {} segmentations". format(
            idx, len(seg_data_by_seq[idx][0]), len(us_data_by_seq[idx][0])))

---

# Divide data into training, validation and testing

In [ ]:
#Create lists of indices for training and validation sets
trainAndVal_idx = list(range(len(seg_data_by_seq)))
trainAndVal_idx = [x for x in trainAndVal_idx if x not in test_idx] #Remove IDs

#Extract and concatenate the segmentations for train / test / val
seg_trainAndVal = np.concatenate(np.take(seg_data_by_seq, trainAndVal_idx), axis=0)
seg_test = np.concatenate(np.take(seg_data_by_seq, test_idx), axis=0)

#Extract and concatenate the ultrasound images for train / test / val
us_trainAndVal = np.concatenate(np.take(us_data_by_seq, trainAndVal_idx), axis=0)
us_test = np.concatenate(np.take(us_data_by_seq, test_idx), axis=0)

#Extract training and validation sets from trainAndVal using 80/20 split
us_train, us_val, seg_train, seg_val = train_test_split(us_trainAndVal, seg_trainAndVal, test_size=0.2, random_state=2)

#Convert validation categorical data to onehot dictated by the number of classes
seg_val_onehot = tf.keras.utils.to_categorical(seg_val, 2)
seg_train_onehot = tf.keras.utils.to_categorical(seg_train, 2)
seg_test_onehot = tf.keras.utils.to_categorical(seg_test, 2)

In [ ]:
print("Trained on {} images, validated on {} images, tested on {} images.".format(us_train.shape[0],
                                                                                 us_val.shape[0],
                                                                                 us_test.shape[0]))

---

# Define IoU loss and metric functions

In [ ]:
from tensorflow.keras import backend as K

def IoU_loss(y_true,y_pred):
    smooth = 1e-12
    intersection = K.sum(y_true[:,:,:,1] * y_pred[:,:,:,1])        #Create intersection
    sum_ = K.sum(y_true[:,:,:,1] + y_pred[:,:,:,1])                #Create union
    jac = (intersection + smooth) / (sum_ - intersection + smooth) #Divide and smooth
    return K.mean(1-jac) #Return 1-IoU so it can be use as a measurement of loss

def IoU(y_true,y_pred):
    smooth = 1e-12
    y_pred_pos = K.round(K.clip(y_pred[:,:,:,1], 0, 1))             #Extract binary mask from probability map
    intersection = K.sum(y_true[:,:,:,1] * y_pred_pos)              #Create union
    sum_ = K.sum(y_true[:,:,:,1] + y_pred[:,:,:,1])                 #Create intersection
    jac = (intersection + smooth) / (sum_ - intersection + smooth)  #Divide and smooth
    return K.mean(jac) #Return the mean jaccard index as IoU

---

# Setup and compile U-Net

In [ ]:
#Get the model object from the aigt unet
model = unet.segmentation_unet(ultrasound_size, 2, filter_multiplier, regularization_rate)

#Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=max_learning_rate, decay=learning_rate_decay),
    loss=IoU_loss,
    metrics=[IoU]
)
#Initialize the training generator
training_generator = generator.UltrasoundSegmentationBatchGenerator(
    us_train,                               #Training US images
    seg_train[:, :, :, 0],                  #Background segmentation of labels
    batch_size,                             #Batch size
    (ultrasound_size, ultrasound_size),     #Image size
    max_shift_factor=max_shift_factor,      #Properties for data augmentation
    min_zoom_factor=min_zoom_factor,
    max_zoom_factor=max_zoom_factor,
    max_rotation_angle=max_rotation_angle
)

#Create a new timestamp for save files
save_timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

model.summary()

---

# Run the UNet and record results

In [ ]:
#Record the time that training starts
training_time_start = datetime.datetime.now()

#Fit the generator to the model
training_log = model.fit_generator(
    training_generator,
    validation_data=(us_val, seg_val_onehot),
    epochs=1,
    verbose=1,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, )
    ])

#Record the time that training stops
training_time_stop = datetime.datetime.now()

#Print training time
print("  Training time: {}".format(training_time_stop-training_time_start))

In [ ]:
# Archive trained model with unique filename based on notebook name and timestamp
model_file_name = notebook_name + current_fold + save_timestamp
model_fullname = os.path.join(output_dir, model_file_name)
model.save(model_fullname)

---

# Graph the Loss and IoU

In [ ]:
def plotTrainingStats(training_history):
    # Plot the loss function
    fig, ax = plt.subplots(1, 1, figsize=(10,6))
    ax.plot(training_history.history['loss'], 'r', label='train')
    ax.plot(training_history.history['val_loss'], 'b' ,label='val')
    ax.set_title("Loss VS Epoch Number")
    ax.set_xlabel(r'Epoch', fontsize=20)
    ax.set_ylabel(r'Loss', fontsize=20)
    ax.legend()
    ax.tick_params(labelsize=20)

    # Plot the accuracy
    fig, ax = plt.subplots(1, 1, figsize=(10,6))
    ax.plot(training_history.history['IoU'], 'r', label='train')
    ax.plot(training_history.history['val_IoU'], 'b' ,label='val')
    ax.set_title("IoU VS Epoch Number")
    ax.set_xlabel(r'Epoch', fontsize=20)
    ax.set_ylabel(r'IoU', fontsize=20)
    ax.legend()
    ax.tick_params(labelsize=20)
    
plotTrainingStats(training_log)

---

# Predict on Test set and print IoU

In [ ]:
#Predict on the test set
y_pred_test  = model.predict(us_test)

#Get the test set IoU and print
iou_test = IoU(seg_test_onehot,y_pred_test)
print("Test Set IoU: {}".format(iou_test))

---

# Show sample images demonstating segmentation

In [ ]:
#Set plotting variables
num_show = 5
num_col = 4
num_vali = 2200
threshold = 0.50

#Randomly generate a set of indices to print
indices = [i for i in range(num_vali)]
sample_indices = sample(indices, num_show)

#Print 4 columns of images
fig = plt.figure(figsize=(18, num_show*5))
for i in range(num_show):
    
    #Plot US image
    a0 = fig.add_subplot(num_show, num_col, i*num_col+1)
    img0 = a0.imshow(np.flipud(us_test[sample_indices[i], :, :, 0].astype(np.float32)))
    a0.set_title("Ultrasound #{}".format(sample_indices[i]))
    
    #Plot original segmentations
    a1 = fig.add_subplot(num_show, num_col, i*num_col+2)
    img1 = a1.imshow(np.flipud(seg_test[sample_indices[i], :, :, 0]), vmin=0.0, vmax=1.0)
    a1.set_title("Segmentation #{}".format(sample_indices[i]))
    
    #Plot Predicted segmentations
    c = fig.colorbar(img1, fraction=0.046, pad=0.04)
    a2 = fig.add_subplot(num_show, num_col, i*num_col+3)
    img2 = a2.imshow(np.flipud(y_pred_test[sample_indices[i], :, :, 1]), vmin=0.0, vmax=1.0)
    a2.set_title("Prediction #{}".format(sample_indices[i]))
    
    #Plot thresholded segmentations
    c = fig.colorbar(img2, fraction=0.046, pad=0.04)
    a3 = fig.add_subplot(num_show, num_col, i*num_col+4)
    img3 = a3.imshow((np.flipud(y_pred_test[sample_indices[i], :, :, 1]) > threshold), vmin=0.0, vmax=1.0)
    c = fig.colorbar(img3, fraction=0.046, pad=0.04)
    a3.set_title("Thresholded #{}".format(sample_indices[i]))